<a href="https://colab.research.google.com/github/Curro-Gonz/Airport-Signs/blob/main/trainingtfg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento y predicción de señales de aeropuerto

Primero, descargamos las dependencias

In [ ]:
!pip install ultralytics==8.0.196
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Obtenemos el directorio en el que vamos a trabajar

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


Importamos cv2_imshow que nos será útil para visualizar los resultados de las operaciones de procesamiento de imágenes.

In [ ]:
from google.colab.patches import cv2_imshow

Importamos ultralytics y YOLO para poder entrenar y predecir el modelo

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/78.2 GB disk)


Cargamos Drive en el colab para poder tener acceso a las carpetas de Drive para luego guardar los resultados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos el link de exportación del *dataset* de Roboflow

Dado que la *api\_key* es privada, se ha tomado la decisión de no compartirla ya que un mal uso de ella puede conllevar la modificación o pérdida del *dataset* sin que el propietario tenga conocimiento de ello. Esto supone que no se podrá descargar el *dataset*.

In [ ]:
%cd {HOME}

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("curro-p77xe").project("airport-signals")
version = project.version(4)
dataset = version.download("yolov8")


/content
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Airport-signals-4 in yolov8:: 100%|██████████| 2762/2762 [00:00<00:00, 4872.33it/s]


Entrenamos nuestro modelo con 100 épocas

In [ ]:
model = YOLO("yolov8n.pt")
results = model.train(data= os.path.join("/content/Airport-signals-4", "data.yaml"), epochs = 100)

100%|██████████| 6.23M/6.23M [00:00<00:00, 178MB/s]
New https://pypi.org/project/ultralytics/8.2.32 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Airport-signals-4/data.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buf

Validamos el modelo con la carpeta de validación

In [ ]:
!yolo task=detect mode=val model='/path/to/best.pt' data='/content/Airport-signals-4/data.yaml'

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs
100% 755k/755k [00:00<00:00, 18.5MB/s]
val: Scanning /content/Airport-signals-4/valid/labels... 102 images, 6 backgrounds, 0 corrupt: 100% 102/102 [00:00<00:00, 1367.33it/s]
val: New cache created: /content/Airport-signals-4/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 7/7 [00:36<00:00,  5.16s/it]
                   all        102        354      0.892      0.875      0.917      0.643
              Airplane        102         64      0.879      0.798      0.888      0.564
             Direction        102         51      0.917      0.866      0.929      0.679
Enhanced taxiway centerline        102         36      0.815      0.917      0.914      0.682
              Location        102         59      0.895      0.863      0.916      0.637
        Mandatory 

Utilizamos nuestro modelo ya entrenado para predecir nuevas imágenes

In [8]:
!yolo task=detect mode=predict model='/path/to/best.pt' conf = 0.5 source='/content/Airport-signals-4/test/images'

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs

WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/55 /content/Airport-signals-4/test/images/172237-13037639_jpg.rf.80fdecebd67caa3dc7b041b690c0a39b.jpg: 640x640 1 Direction, 17.3ms
image 2/55 /content/Airport-signals-4/test/images/20231003_054930000_iOS_MOV-2_jpg.rf.e2d83c87752d2255d0ec888302d75db2.jpg: 640x640 5 Airplanes, 1 Enhanced taxiway centerline, 1 Runway holding position, 7.4ms
image 3/55 /content/Airport-signals-4/test/images/20231003_054930000_iOS_MOV-3_jpg.rf.b29dc509bc156987802cdbbc0c9baed4.jpg: 640x640 5 Airplanes, 1 Enhanced taxiway centerline, 1 Runway holding position, 7.3ms
image 4/55 /content/Airport-signals-4/test/images/20231003_094601000_iOS_MOV-1_jpg.rf.d09db8bd1401dff834ccc32b4fdb9d6e.jpg: 640x640 1 Location, 9.4ms
image 5/55 /content/Airport-signals-4/test/images/20231003_094601000_iOS_MOV-23_jpg

Guardamos los resultados en Drive

In [ ]:
import shutil
# Ruta del directorio que quieres copiar
source_dir = "/content/runs"
# Ruta del directorio de destino en Google Drive
dest_dir = "/content/drive/MyDrive"

# Copiar el directorio
shutil.copytree(source_dir, dest_dir)

'/content/drive/MyDrive/TFG_Google_Colab/runs/detect/predict3'